In [1]:
!pip install streamlit -q
!npm install localtunnel -g -s
!pip install librosa matplotlib seaborn tensorflow -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 137.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 146.0 MB/s eta 0:00:00


In [2]:
# HÜCRE 1: Kurulum ve Bağlantı
from google.colab import drive
import os

# 1. Drive'ı Bağla
drive.mount('/content/drive')
print("Drive bağlandı, paketler yüklendi.")

Mounted at /content/drive
Drive bağlandı, paketler yüklendi.


In [3]:
# HÜCRE 2: app.py Dosyasını Oluşturma
%%writefile app.py

import streamlit as st
import numpy as np
import librosa
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import load_model
import os

MODEL_PATH = '/content/drive/MyDrive/Colab Notebooks/SER_Projesi/SER_Model_CNN_Acc_89.keras'

EMOTIONS = ['angry', 'calm', 'disgust', 'fearful', 'happy', 'neutral', 'sad', 'surprised']

# --- MODEL YÜKLEME ---
@st.cache_resource
def load_ser_model():
    if not os.path.exists(MODEL_PATH):
        st.error(f"HATA: Model dosyası bulunamadı!\nYol: {MODEL_PATH}\nLütfen kod içindeki yolu kontrol et.")
        return None
    model = load_model(MODEL_PATH)
    return model

model = load_ser_model()

if model:
    st.success(" Yapay Zeka Beyni Yüklendi ve Hazır!")

# --- SES İŞLEME ---
def preprocess_audio(audio_path):
    # 1. Yükle
    data, sr = librosa.load(audio_path, duration=3, offset=0.5, res_type='kaiser_fast')

    # 2. 3 Saniyeye Sabitle
    target_len = 3 * 22050
    if len(data) > target_len:
        data = data[:target_len]
    else:
        padding = target_len - len(data)
        data = np.pad(data, (0, padding), 'constant')

    # 3. Özellik Çıkar
    mfcc = librosa.feature.mfcc(y=data, sr=sr, n_mfcc=40)
    mfcc_processed = np.mean(mfcc.T, axis=0)

    # 4. Boyutlandır
    return np.expand_dims(np.expand_dims(mfcc_processed, axis=0), axis=2)

# --- ARAYÜZ ---
st.title(" Duygu Analiz Dedektifi")
st.write("Bu proje, ses tonundan duygu durumunu %88 doğrulukla tahmin eder.")

uploaded_file = st.file_uploader("Test etmek için bir ses dosyası (.wav) yükleyin:", type=["wav"])

if uploaded_file is not None and model is not None:
    st.audio(uploaded_file, format='audio/wav')

    if st.button("Duyguyu Analiz Et "):
        with st.spinner('Yapay Zeka düşünüyor...'):
            with open("temp.wav", "wb") as f:
                f.write(uploaded_file.getbuffer())

            processed_data = preprocess_audio("temp.wav")
            prediction = model.predict(processed_data)
            predicted_index = np.argmax(prediction)
            confidence = np.max(prediction)
            predicted_emotion = EMOTIONS[predicted_index]

            # Sonuç Alanı
            st.divider()
            col1, col2 = st.columns(2)

            with col1:
                st.subheader("Tespit Edilen Duygu:")
                st.markdown(f"<h1 style='color: #FF4B4B;'>{predicted_emotion.upper()}</h1>", unsafe_allow_html=True)
                st.info(f"Güven Skoru: %{confidence*100:.2f}")

            with col2:
                st.subheader("Olasılık Dağılımı:")
                fig, ax = plt.subplots(figsize=(4, 3))
                sns.barplot(x=EMOTIONS, y=prediction[0], palette='viridis', ax=ax)
                plt.xticks(rotation=45, fontsize=8)
                st.pyplot(fig)

            # Yorum
            if predicted_emotion in ['angry', 'fearful', 'disgust', 'sad']:
                st.warning(" Negatif veya stresli bir duygu algılandı.")
            elif predicted_emotion in ['happy', 'surprised']:
                st.success(" Pozitif veya yüksek enerjili bir duygu!")
            else:
                st.success(" Sakin veya nötr bir durum.")

Writing app.py


In [10]:
!pip install resampy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 96.8 MB/s eta 0:00:00


In [ ]:
# HÜCRE 3 : Uygulamayı Canlıya Al

# 1. Önce ngrok kütüphanesini kuralım
!pip install pyngrok -q

from pyngrok import ngrok

NGROK_TOKEN = "YOUR_API_KEY"

# 3. Token'ı sisteme tanıt
!ngrok config add-authtoken {NGROK_TOKEN}

# 4. Tüneli Aç
public_url = ngrok.connect(8501).public_url
print(f" CANLI, KOPMAYAN LINK: {public_url}")

# 5. Uygulamayı Başlat
!streamlit run app.py &>/content/logs.txt

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
 CANLI, KOPMAYAN LINK: https://wambly-evette-unmiasmal.ngrok-free.dev
